# TotalCapture pre-processing (to pair it with AMASS SMPL ground truth)

Start by getting a list of the TotalCapture recordings that have a .npz linked to it, and that match in length (e.g. some .npz's have less frames than the videos, and are thus disconsidered)

Then, with this list saved in a dictionary placed at the working directory,

In [ ]:
# To be ran at the directory holding both AMASS and TotalCapture
# (e.g. place AMASS inside the data folder in TotalCaptureToolbox)
# root
#  data
#   AMASS
#   images

# Create a dictionary containing the name of the action in TotalCapture-Toolbox and AMASS notation,
# and whether there is a .npz file for it and if it's a match (frames to number of poses)

import glob
import os
import pickle
import numpy as np

subject_map = {1: 's1', 2: 's2', 3: 's3', 4: 's4', 5: 's5'}
action_map = {1: 'rom', 2: 'walking', 3: 'acting', 4: 'running', 5: 'freestyle'}

def to_npz_name(total_capture_path):
    _, short_path = os.path.split(total_capture_path) # Throw out all path but 's_xx_act_xx_subact_xx_ca_xx'
    npz_style_name = os.path.join(subject_map[int(short_path[3])], action_map[int(short_path[10])] + short_path[-7] + '_poses.npz')
    return npz_style_name

npz_path = 'data/AMASS/'
npz_paths = glob.glob(npz_path+'*/*') # Gets all .npz's names from amass

npz_short_paths = []

for i, path in enumerate(npz_paths):
    head, action = os.path.split(npz_paths[i])
    _, subject = os.path.split(head)
    short_npz = os.path.join(subject, action)
    npz_short_paths.append(short_npz)
    

image_path = 'data/images/'
img_paths = glob.glob(image_path+'*') # Gets all actions, subactions and camera folder names
img_paths = [path for path in img_paths if os.path.isdir(path)]

short_paths = []
frames_in_path = []
npz_style_names = []
has_npzs = []
matches_frames = []
n_poses = []

action_camera_dict = {}

for i, path in enumerate(img_paths):
    print('Found path:', path)
    n_frames = (len(os.listdir(path))) # Counts how many images are inside the folder
    _, short_path = os.path.split(img_paths[i]) # Throw out all path but 's_xx_act_xx_subact_xx_ca_xx'
    npz_style_name = to_npz_name(path)
    has_npz = npz_style_name in npz_short_paths

    match_frames = False
    n_pose = None
    
    if has_npz:
        arr = np.load(os.path.join(npz_path, npz_style_name))
        n_pose = arr['poses'].shape[0]
        match_frames = (n_pose == n_frames)
    
    action_camera_dict[short_path] = {'n_frames': n_frames,
                                   'has_npz': has_npz,
                                   'npz_style_name': npz_style_name,
                                   'n_poses': n_pose,
                                   'is_a_match': match_frames}

# Uncomment to store .pkl
# The created .pkl should have 296 entries with 'has_npz' = True, since that's 37 AMASS files * 8 cameras.

f = open('action_camera_dict.pkl', 'wb')
pickle.dump(action_camera_dict, f)
f.close()

# Start of pre-processing code

After getting a dictionary containing the actions in Total capture with the code above, we extract the image name, center and scale from the annotations in the dictionary that have 'is_a_match' set to True. This means only actions & cameras that exactly match the number of frames to the corresponding AMASS .npz are used.

In [10]:
AMASS_TOTAL_CAPTURE_ROOT = 'data/AMASS/'

In [11]:
def img_to_npz_name(total_capture_img_path):
    # Returns the frame number of the image and the npz style name of the action
    
    import os
    
    subject_map = {1: 's1', 2: 's2', 3: 's3', 4: 's4', 5: 's5'}
    action_map = {1: 'rom', 2: 'walking', 3: 'acting', 4: 'running', 5: 'freestyle'}
    
    folder, file = os.path.split(total_capture_img_path) # Separate image and folder
    _, short_path = os.path.split(folder) # Throw out everything from the folder except 's_xx_act_xx_subact_xx_ca_xx'
    
    npz_style_name = os.path.join(subject_map[int(short_path[3])], action_map[int(short_path[10])] + short_path[-7] + '_poses.npz')
    frame = file[:-4]
    frame = int(frame)
    
    return npz_style_name, frame

In [12]:
def is_from_current_npz(total_capture_img_path, npz_file):
    return img_to_npz_name(total_capture_img_path)[0] == short_npz_name(npz_file)

In [13]:
import pickle

filename = 'action_camera_dict.pkl'

with open(filename, 'rb') as f:
    action_camera_dict = pickle.load(f, encoding='latin1')

def is_valid_image(image):
    folder, _ = os.path.split(image)
    if action_camera_dict[folder]['is_a_match']:
        return True

In [14]:
def short_npz_name(file_name):
    short_name = file_name[len(AMASS_TOTAL_CAPTURE_ROOT):]
    return short_name

# Import the relevant .pkl with annotations from TotalCapture (Either train or validation)

In [65]:
import pickle

#filename = 'totalcapture_train.pkl'
filename = 'totalcapture_validation.pkl'

with open(filename, 'rb') as f:
    pkl_data = pickle.load(f, encoding='latin1')

# Set the sampling rate

In [66]:
SAMPLING_RATE = 0.1 # Set the probability we will use an image (0.1 to downsample ten times)

In [67]:
import numpy as np
import glob
import os
import sys

imgnames_ = []
centers_ = []
scales_ = []

poses_ = []
shapes_ = []
genders_ = []

npz_files = glob.glob(AMASS_TOTAL_CAPTURE_ROOT+'*/*')

np.random.seed(42)

i=0
for file in npz_files:
    with np.load(file, mmap_mode = 'r') as f:
        for d in pkl_data:
            if is_valid_image(d['image']):
                if np.random.random() < SAMPLING_RATE:
                    if img_to_npz_name(d['image'])[0] == short_npz_name(file):

                        imgname = d['image']

                        npz_style_name, frame = img_to_npz_name(imgname)

                        center = d['center']
                        scale = max(d['scale'])

                        pose = np.copy(f['poses'][frame][:72]) # Loads the first 72 of thee 156-size pose from AMASS (because it includes hand pose)
                        shape = np.copy(f['betas'][:10]) # Loads the first 10 of the 16 betas
                        gender = (str(f['gender'])[0]) # Loads the first letter of the gender string

                        imgnames_.append(imgname)
                        centers_.append(center)
                        scales_.append(scale)
                        poses_.append(pose)
                        shapes_.append(shape)
                        genders_.append(gender)


                        if (i % 1000==0):
                            print('processing ', file, i, 'total frames')
                        i+=1

        print('processed', file)

out_path = '.'
out_file = os.path.join(out_path, filename[:-4] + str(SAMPLING_RATE) + '.npz')
np.savez(out_file, imgname=imgnames_,
                       center=centers_,
                       scale=scales_,
                       pose=poses_,
                       shape=shapes_,
                       gender=genders_)

['data/AMASS/s5/rom3_poses.npz', 'data/AMASS/s5/walking2_poses.npz', 'data/AMASS/s5/freestyle1_poses.npz', 'data/AMASS/s5/freestyle3_poses.npz', 'data/AMASS/s2/rom3_poses.npz', 'data/AMASS/s2/walking3_poses.npz', 'data/AMASS/s2/walking2_poses.npz', 'data/AMASS/s2/walking1_poses.npz', 'data/AMASS/s2/rom2_poses.npz', 'data/AMASS/s2/rom1_poses.npz', 'data/AMASS/s2/acting1_poses.npz', 'data/AMASS/s2/acting3_poses.npz', 'data/AMASS/s2/acting2_poses.npz', 'data/AMASS/s2/freestyle2_poses.npz', 'data/AMASS/s4/rom3_poses.npz', 'data/AMASS/s4/walking2_poses.npz', 'data/AMASS/s4/freestyle1_poses.npz', 'data/AMASS/s4/freestyle3_poses.npz', 'data/AMASS/s1/rom3_poses.npz', 'data/AMASS/s1/walking3_poses.npz', 'data/AMASS/s1/walking2_poses.npz', 'data/AMASS/s1/walking1_poses.npz', 'data/AMASS/s1/rom2_poses.npz', 'data/AMASS/s1/freestyle1_poses.npz', 'data/AMASS/s1/rom1_poses.npz', 'data/AMASS/s1/acting1_poses.npz', 'data/AMASS/s1/acting3_poses.npz', 'data/AMASS/s1/freestyle3_poses.npz', 'data/AMASS/s1

In [68]:
npz = np.load(out_file)

In [69]:
len(npz['imgname'])

10000

# Save imgnames to a .txt for batch copy

In [71]:
np.savetxt('copy_list_subsampled.txt', npz['imgname'], fmt='%s')